In [ ]:
import logging
import time
from datetime import datetime

import geohash
import numpy as np
import pandas as pd
import geopandas as gpd
import dask
import dask.dataframe as dd
from distributed import LocalCluster, Client

In [ ]:
cluster = LocalCluster(n_workers=4, threads_per_worker=6)#silence_logs=logging.ERROR)
client = Client(cluster)
client

In [3]:
df = dd.read_parquet('../data/resaved-gps-points', engine='pyarrow')
# df = df.partitions[1217:].repartition(npartitions=8).persist()
df.head()

,latitude,longitude
index,,
0,-90.0,-39.374215
1,-90.0,-5.624215
2,-90.0,64.688241
3,-90.0,90.000000
4,-90.0,98.438280


In [4]:
def geohash_world(df):
    if 'geohash' not in df.columns:
        df['geohash'] = ''
    lat90indices = (df.latitude == 90) 
    df.loc[lat90indices, 'geohash'] = 'zzzzzzzzzzzz' # geohash.encode fails if lat==90
    
    valid_indices = (df.longitude.between(-180, 180)) & (df.latitude >= -90) & (df.latitude < 90)
    df.loc[valid_indices, 'geohash'] = df.loc[valid_indices, :].apply(lambda row: geohash.encode(row.latitude, row.longitude), axis=1)
    return df

In [ ]:
%%time 
t0 = time.time()
df = df.map_partitions(geohash_world, meta={'latitude': float, 'longitude': float, 'geohash': 'object'})
df.to_parquet('../data/geohashed_gps_points5')
dt_hr = (time.time() - t0)/60/60

In [ ]:
with open(f'{datetime.now()}_geohash_time4.csv', 'w') as f:
    f.write(f'dt_hr, {dt_hr}')